# Optimizing Strategy Parameters

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [2]:
import pandas as pd

df = pd.read_excel('data/INTC_subset_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2020-01-02,60.240002,60.970001,60.220001,60.840000,18056000,-1.231284,DOWN
2020-01-03,59.810001,60.700001,59.810001,60.099998,15293900,-0.283661,DOWN
2020-01-06,59.590000,60.200001,59.330002,59.930000,17755200,-1.696929,DOWN
2020-01-07,59.779999,59.799999,58.889999,58.930000,21876100,0.067833,UP
2020-01-08,58.889999,59.320000,58.520000,58.970001,23133500,0.556489,UP
...,...,...,...,...,...,...,...
2024-09-26,24.280001,24.420000,23.250000,23.920000,95416900,-0.041824,DOWN
2024-09-27,24.160000,24.660000,23.700001,23.910000,85883300,-1.918162,DOWN
2024-09-30,23.740000,23.950001,23.090000,23.459999,66308200,-3.393559,DOWN


# Simple Investment Strategy

### Create Strategy class

In [3]:
from backtesting import Strategy, Backtest

In [4]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [5]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [6]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [7]:
model_dt.predict(df_explanatory)

array([-1.019789  , -1.019789  , -1.019789  , ..., -3.39355898,
       -1.33988902, -0.58400341])

In [8]:
results = bt.run()

### Interpret backtesting results

In [9]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2020-01-02 00:00:00
End,2024-10-02 00:00:00
Duration,1735 days 00:00:00
Exposure Time [%],98.913043
Equity Final [$],47657.826326
Equity Peak [$],50285.326326
Return [%],376.578263


## Parametrize the Investment Strategy

### Create Strategy class

In [10]:
# Summary:
# This is a trading strategy class based on a regression model. It predicts the price movement 
# for the next day and decides whether to buy or sell based on predefined limits.
# If the predicted price is above a certain limit (limit_buy), the strategy buys, 
# provided it hasn't already bought. If the predicted price is below a certain limit (limit_sell), 
# it sells, provided it has already bought. The strategy avoids overbuying or overselling 
# by using the 'already_bought' flag.

class Regression(Strategy):
    
    limit_buy = 1  # Set the limit above which the strategy will trigger a buy action
    limit_sell = -5  # Set the limit below which the strategy will trigger a sell action
    
    def init(self):
        # Initialize the model and the state of whether an asset has been bought
        self.model = model_dt  # Load the pre-trained regression model
        self.already_bought = False  # Track whether the strategy has already bought an asset
    
    def next(self):
        # Get today's data (the latest available) and make a prediction for tomorrow
        explanatory_today = self.data.df.iloc[[-1], :]  # Extract the latest row of data
        forecast_tomorrow = self.model.predict(explanatory_today)[0]  # Predict tomorrow's price
        
        # Buy condition: forecast is higher than limit_buy and no prior purchase was made
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()  # Trigger a buy action
            self.already_bought = True  # Set flag to indicate that a purchase has been made
        
        # Sell condition: forecast is lower than limit_sell and a purchase was made earlier
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()  # Trigger a sell action
            self.already_bought = False  # Reset the flag to indicate the asset has been sold
        
        # If neither condition is met, do nothing
        else:
            pass


### Create Backtest class

In [11]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [12]:
list_limits_buy = list(range(0, 11, 1))

In [13]:
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
list_limits_sell = list(range(0, -11, -1))

In [15]:
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [16]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

CPU times: total: 43 s
Wall time: 1min 8s


### [ ] Interpret backtesting results

In [17]:
# Extract the second item from 'results', which contains the data for the heatmap
results_heatmap = results[1]

In [18]:
# Reset the index of the DataFrame to convert index into regular columns
df_results_heatmap = results_heatmap.reset_index()

In [19]:
# Pivot the DataFrame to create a 2D table where 'limit_buy' is the row index,
# 'limit_sell' is the column index, and 'Return [%]' is the values being displayed
dff = df_results_heatmap.pivot(
    index='limit_buy', columns='limit_sell', values='Return [%]')
dff

In [21]:
# Sort the columns (limit_sell) in descending order
dff.sort_index(axis=1, ascending=False)

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,66641.177357,19630.144916,3798.309709,1330.537599,1100.539619,496.158036,57.273568,11.614849,-3.129214,-3.129214,-20.370664
1,12748.331970,5909.907902,3172.793751,1195.048456,1079.564659,376.578263,9.658269,-18.036657,-35.060556,-35.060556,-46.577858
2,3837.079967,5028.156528,4552.863775,1553.207500,1882.832769,363.148333,12.744782,-17.222307,-34.594555,-34.594555,-46.126425
3,1692.187104,2859.880862,2411.160927,1216.109942,1774.971190,325.821912,-12.104173,-46.030668,-56.996602,-56.996602,-64.604621
4,1111.664311,1196.024347,1092.284364,710.707009,1079.016459,677.278474,439.055901,231.162555,37.409580,37.409580,24.545774
5,521.548728,531.102523,498.113739,443.993613,644.730517,523.379664,539.524712,240.551372,240.551372,240.551372,208.622968
6,248.669818,172.880425,160.185214,140.028278,174.584845,133.972842,420.023253,222.125599,222.125599,222.125599,208.622968
7,238.287912,164.795878,161.498992,141.157352,179.920880,108.009616,409.946778,266.194142,266.194142,266.194142,249.483634
8,80.396415,80.396415,80.396415,72.479390,69.257437,61.145269,295.337274,282.912228,282.912228,282.912228,326.257023


## DataFrame heatmaps for better reporting

In [22]:
# Style the DataFrame: Sort the columns in descending order, set precision to 0 decimal places,
# and apply a background gradient based on the values in the DataFrame

# Format the numbers with no decimals and Apply background color gradient based on values
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient()

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,66641,19630,3798,1331,1101,496,57,12,-3,-3,-20
1,12748,5910,3173,1195,1080,377,10,-18,-35,-35,-47
2,3837,5028,4553,1553,1883,363,13,-17,-35,-35,-46
3,1692,2860,2411,1216,1775,326,-12,-46,-57,-57,-65
4,1112,1196,1092,711,1079,677,439,231,37,37,25
5,522,531,498,444,645,523,540,241,241,241,209
6,249,173,160,140,175,134,420,222,222,222,209
7,238,165,161,141,180,108,410,266,266,266,249
8,80,80,80,72,69,61,295,283,283,283,326


In [23]:
# Import numpy to use for finding min and max of DataFrame for the gradient scale
import numpy as np

In [24]:
# Style the DataFrame again: Sort columns in descending order, format precision,
# and apply a background gradient where the gradient is scaled using the minimum and maximum values

# Format the numbers with no decimals and Apply background gradient with min-max scaling
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,66641,19630,3798,1331,1101,496,57,12,-3,-3,-20
1,12748,5910,3173,1195,1080,377,10,-18,-35,-35,-47
2,3837,5028,4553,1553,1883,363,13,-17,-35,-35,-46
3,1692,2860,2411,1216,1775,326,-12,-46,-57,-57,-65
4,1112,1196,1092,711,1079,677,439,231,37,37,25
5,522,531,498,444,645,523,540,241,241,241,209
6,249,173,160,140,175,134,420,222,222,222,209
7,238,165,161,141,180,108,410,266,266,266,249
8,80,80,80,72,69,61,295,283,283,283,326
